# Word Embeddings Workshop

March 19, 2021

Instructor: Connor Gilroy, Department of Sociology, University of Washington  
TA: Nga Than, Sociology program, CUNY Graduate Center

## Introduction

The goal of this workshop is to give you an intuitive and practical introduction to what word embeddings are and what they can be used for in the social sciences.

You can find a short introductory slide deck [here](https://docs.google.com/presentation/d/1V4SaADerFMph9wB7pES76vYUWhIzvQ_LMpSyAIB6f_o/edit?usp=sharing).

## Packages used

This workshop primarily teaches and uses the **gensim** package. The main corpus (the 20 Newsgroups data set) comes from scikit-learn. 

An optional part of the tutorial also uses the *whatlies* package. For the exercises, you may find it helpful to load some common Python data science packages as well, if you have those installed in your environment.

In [ ]:
import gensim
import sklearn

# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import altair as alt

## 1. Using a pretrained model (GloVe)

Things to know about a pretrained model: 

- What's the overall modeling approach?
- What vector size?
- What vocabulary size?
- What other parameters might affect substantive results? 
- What data was it trained on?

Some of those things will be well-documented or obvious, some won't be.

The GloVe models are originally documented [on a project page](https://nlp.stanford.edu/projects/glove/) from the Stanford NLP Group, but the gensim package also stores data for these and other models [in the gensim-data repository](https://github.com/RaRe-Technologies/gensim-data). We'll download and load a small pretrained model from the latter; click the link to read more about it. 

This model is around 66 MB. It's relatively small because each word is represented by a vector of only 50 numbers. Larger vectors (150-300 dimensions) are more common in practice.

In [ ]:
import gensim.downloader as api
glove_embeddings = api.load('glove-wiki-gigaword-50')

We can confirm that these vectors do indeed have 50 dimensions:

In [ ]:
glove_embeddings.vector_size

How many words are in the vocabulary?

In [ ]:
len(glove_embeddings.vocab)

This is a 400,000 x 50 matrix, which can be accessed through `glove_embeddings.vectors`: 

In [ ]:
glove_embeddings.vectors.shape

If a word is in the vocabulary, you can extract its embedding from the model like this:

In [ ]:
glove_embeddings["society"]

Differerent word, different vector: 

In [ ]:
glove_embeddings["individual"]

0.555 corresponds to 0.208, 0.493 corresponds to 0.895, and so on. (What do those individual positions mean? Not much on their own!)

This is what unlocks the key innovation of word embeddings: we can calculate the similarity or distance between two words using their vector representations. This is usually done with a metric called **cosine similarity**, which ranges from -1 to 1 (=perfectly similar).

In [ ]:
glove_embeddings.similarity("society", "individual")

Of course, a word is exactly similar to itself: 

In [ ]:
glove_embeddings.similarity("society", "society")

In [ ]:
# try out calculating the similarities between a few other pairs of words here!

(Technical aside: cosine similarity is the dot product of two vectors, divided by the L2-norm for each vector.)

In [ ]:
from numpy import dot
from numpy.linalg import norm

In [ ]:
(dot(glove_embeddings["society"], glove_embeddings["individual"]) / 
    (norm(glove_embeddings["society"], ord=2) * norm(glove_embeddings["individual"], ord=2)))

To understand how a model respresents a particular word, we can look at which words are the most similar to it according to that model.

This is the foundation for making substantive claims about meaning: for instance, what a word means in a given set of documents, or how the meaning of a word has changed over time. 

Here are the 10 most similar words to "society" in the GloVe vocabulary: 

In [ ]:
glove_embeddings.most_similar("society", topn=10)

And the 10 most similar words to "individual":

In [ ]:
glove_embeddings.most_similar("individual", topn=10)

Relations of similarity are the basic operation, but we're not limited to the individual word vectors we started with. We can do algebra on those vectors to build up new vectors that represent more complex meanings. For instance, we can take the *difference* between two vectors, in order to represent a binary opposition between them.  

Researchers build on this basic idea to create more robust vectors representing concepts that can be thought of in a binary way. Kozlowski et al 2019, for example, represent multiple dimensions of social class by averaging different pairs of antonyms. They then show how the associations between these dimensions change over the course of the 20th century.

The next example constructs an opposition between "society" and "individual" -- this will give us the words that are the closest to the "society" end of that dimension, or the "individual" end.

(You might be able to tell that these vectors were trained on Wikipedia entries, because the vocabulary includes many rare words -- we might get more interesting and meaningful results if we filtered the 400,000-word vocabulary first.)

In [ ]:
glove_embeddings.most_similar(positive=["society"], negative=["individual"])

In [ ]:
glove_embeddings.most_similar(positive=["individual"], negative=["society"])

What's happening here, behind the scenes, is that one vector is being subtracted from the other. 

This creates a new vector, like this:

In [ ]:
glove_embeddings["society"] - glove_embeddings["individual"]

You might want to use that derived vector as part of your model, so here's how to add it into the model's overall set of vectors. 

(This next part is a bit technical and not substantively interesting, so I'll gloss over the details. For an alternative, check out the whatlies package below.) 

In [ ]:
diff = glove_embeddings["society"] - glove_embeddings["individual"]
glove_embeddings.add("society - individual", diff)
glove_embeddings["society - individual"]

In order for `most_similar()` to actually work with the "society - individual" vector, there's one more necessary step to run, in the next cell. 

(Why? Adding the vector doesn't automatically create an L2-normalized version of it, which `most_similar()` needs. `init_sims()` will recalculate, but only if the rest of the normed vectors are removed first.) 

In [ ]:
glove_embeddings.vectors_norm = None
glove_embeddings.init_sims()

Now we can query this single vector and get the same results: 

In [ ]:
glove_embeddings.most_similar(positive=["society - individual"])

In [ ]:
glove_embeddings.most_similar(negative=["society - individual"])

### Exercise

(Adapted from Kozlowski et al 2019) Make a list of sports. Construct a "rich" vs "poor" dimension to proxy social class. Which sports have the strongest class associations in either direction?

In [ ]:
# add to this list!
sports = ["hockey", "baseball", ] 

In [ ]:
# create a rich - poor vector 
# then add it to the GloVe vectors

In [ ]:
# calculate similarity between class dimension and each sport
sport_similarities = []
for sport in sports:
    pass # replace with code to calculate similarity - you 

## 2. Creating a locally trained model (Word2Vec)

If you're interested in word associations in a particular collection of texts, or if you suspect that those texts use language really differently from the sources pretrained models derive from (largely different kinds of contemporary internet data), then you might want to train your own model.

### Getting a corpus (20 Newsgroups)

To train a new model on a corpus, we need a corpus. We'll use the [20 Newsgroups data set](http://qwone.com/~jason/20Newsgroups/), which was created in 1995 for use in text-related machine learning research. Because the posts are partitioned into different groups, it's mainly used in classification and clustering applications. For an interesting contemporary example, which uses clusters of pretrained embeddings analogously to topic models, see [Sia et al 2020](http://arxiv.org/abs/2004.14914). 

(Sociologically, I find Usenet interesting as a historical precursor to contemporary online communities. Check out Nancy Baym's 1994 paper or her 2000 book if you'd like more ethnographic context, or Avery Dame-Griff 2019 if you'd like a historically-informed computational analysis.)

Running this code will download the 20 Newsgroups data set:

In [ ]:
from sklearn.datasets import fetch_20newsgroups

twenty_newsgroups = fetch_20newsgroups(data_home="data",
                                       subset="all",
                                       shuffle=False,
                                       remove=('headers', 'footers', 'quotes'),
                                       download_if_missing=True)

The data set has about 18000 posts. The way the `remove` argument parses the posts is approximate, so some of those posts wind up being empty strings.

In [ ]:
len(twenty_newsgroups.data)

Let's look at the data by printing out a few posts at random.

In [ ]:
print(twenty_newsgroups.data[0])

In [ ]:
print(twenty_newsgroups.data[5])

In [ ]:
# Look at a few more!

You might notice some obvious themes or topics in the text. These are the groups included in the data set: 

In [ ]:
twenty_newsgroups.target_names

Preprocessing is an important (and sometimes underappreciated) step in text analysis, which [can have downstream consequences](https://github.com/matthewjdenny/preText). 

We'll preprocess eacg Usenet post pretty minimally -- remove punctuation and special characters, lowercase, and tokenize it into a list of words: 

In [ ]:
from gensim.utils import simple_preprocess

print(simple_preprocess(twenty_newsgroups.data[5], min_len=1, max_len=20))

In [ ]:
preprocessed_docs = []
for doc in twenty_newsgroups.data:
    preprocessed_docs.append(simple_preprocess(doc, min_len=1, max_len=20))

### Fitting the model

That was a lot of data work, but now we're ready to train a word2vec model. 

We'll use skip-gram (`sg=1`) with negative sampling (`negative=5`). We'll use a small vector size (`size=50`) because it's faster to train. We'll make a few passes over the data set (`iter=10`), and we'll ignore any words appearing less than 5 times total (`min_count=5`). 

To start with, let's try a small context window around each word (`window=5`). 

**Note:** This small vector size, combined with this small window size, isn't something we'd expect to yield a great model, but it's relatively quick to train. (On my laptop, that means less than two minutes -- it might vary for your machine. `workers=3` means that it runs on three processes.)

In [ ]:
from gensim.models import Word2Vec

w2v_model1 = Word2Vec(sentences=preprocessed_docs, 
                      size=50, 
                      window=5, 
                      min_count=5,
                      workers=3,
                      sg=1, 
                      hs=0, 
                      negative=5,
                      iter=10)

Here are some basic properties of the trained model. Note that the embeddings themselves are accessed through the `wv` property. 

In [ ]:
w2v_model1.corpus_count

In [ ]:
w2v_model1.total_train_time

In [ ]:
w2v_model1.wv.vectors.shape

The vector for a given word will be a different string of numbers from the previous GloVe model. (And there's no reason for the dimensions to correspond, either!)

In [ ]:
w2v_model1.wv["society"]

Instead, we'll look at the most similar words again, to get a qualitative impression for whether this model is encoding a meaning similar to the GloVe model. 

In [ ]:
w2v_model1.wv.most_similar("society")

Again, for the word "individual": 

In [ ]:
w2v_model1.wv.most_similar("individual")

We won't repeat every step from Part 1 above. 

Instead, let's experiment with changing one parameter for training the model, the context window around each word. We'll fit a new model with `window=10`. 

In [ ]:
w2v_model2 = Word2Vec(sentences=preprocessed_docs, 
                      size=50, 
                      window=10, 
                      min_count=5,
                      workers=3,
                      sg=1, 
                      hs=0, 
                      negative=5,
                      iter=10)

In [ ]:
w2v_model2.total_train_time

In [ ]:
w2v_model2.wv.most_similar("society")

Qualitatively, smaller windows (e.g. 5) tend to encode more syntactic similarities (words that are substitutes), whereas larger windows (e.g. 50) encode more semantic similarities (words that are topically similar). See Rodriguez and Spirling 2020 or [this PyData talk from 2017](https://www.youtube.com/watch?v=tAxrlAVw-Tk&t=648s).

### Exercise

(Adapted from Rodriguez and Spirling 2020) How correlated are those two word2vec models? Pick a focal word like "society" and calculate cosine similarity between that word and every word in the entire model vocabulary, for each model. Then calculate the correlation between those two measures.

(You might use `most_similar()` with the appropriate value for `topn`, or `similarity()` with a for-loop. The first is probably more efficient, but then you'll need to sort or join the two lists based on the words. Python data science packages like pandas may be helpful.) 

---

On your own, after the workshop: 

- Try a larger vector size (e.g. 100, 150, 200, 300)
- Try a larger window size (e.g. 20, 50) 
- Try reducing min_count (e.g. from 5 to 2)

**Note: all of these parameter changes make the model take longer to train!**

In [ ]:
from scipy.stats import pearsonr

In [ ]:
# write your code here

## 3. Visualizing embeddings (whatlies) [OPTIONAL]

The [whatlies package](https://rasahq.github.io/whatlies/) is relatively new (Warmerdam et al 2020); it's meant to facilitate exploring and visualizing embeddings.

It works as a wrapper around embeddings from different packages, including gensim. We'll turn the word2vec model from Part 3 into a `whatlies.EmbeddingSet`.

In [ ]:
from whatlies import Embedding, EmbeddingSet

In [ ]:
emb_w2v = EmbeddingSet.from_names_X(names=w2v_model1.wv.index2word, 
                                    X=w2v_model1.wv.vectors)

whatlies has built-in functions for plotting things like cosine similarities (built on the Python visualization packages matplotlib and altair):

In [ ]:
emb_w2v.embset_similar("society", n=10, metric="cosine").plot_similarity()

In [ ]:
emb_w2v.embset_similar("individual", n=10, metric="cosine").plot_similarity()

It also implements vector algebra on embeddings in a nice way: 

In [ ]:
emb_w2v.embset_similar(emb_w2v["society"] - emb_w2v["individual"]).plot_similarity()

We can plot the similarities of different vectors against each other on the axes as well:

In [ ]:
(emb_w2v["society", "nation", "individual", "exclusive"]
 .plot(x_axis="society", 
       y_axis="individual", 
       axis_metric="cosine_similarity"))

Or, we can transform the local vector space. Mouse over the points below to see the words.

(You could, of course, use any dimensionality reduction technique on the entire vector space as well.)

In [ ]:
from whatlies.transformers import Pca

In [ ]:
(emb_w2v
 .embset_similar("society", n=100, metric="cosine")
 .transform(Pca(2))
 .plot_interactive(annot=False))

### Exercise

Write code to convert the GloVe embeddings from a gensim `KeyedVectors` object to a whatlies `EmbeddingSet`. (Note: you don't need to use the `wv` attribute. Instead, access `glove_embeddings.index2word` directly. That's what the warning message "use self" means.)

Then, select some neighborhood of words (maybe a larger one than n=100) around a focal word (like "society"), transform those vectors with PCA, and plot them. 


In [ ]:
# convert glove embeddings to an EmbeddingSet

In [ ]:
# select, transform, and plot

## Wrapping up

### Main takeaways

- Word embeddings have become one tool in the computational text analysis toolkit.
- They've been successfully applied in the social sciences, in cases where the relational meanings of words and concepts are of substantive interest.

### What to do next

- Keep playing around in this notebook, with different words and parameters!
- Apply a pretrained model to or train a local model on your own corpus.
- Have a look at the code for some of the papers listed below.
- Read through my [experimental notebooks](https://ccgilroy.github.io/community-discourse/). I wrote those notebooks for myself, to explore how these methods could be applied to a topic I'm interested in, but you might find them useful.

The main additional topic I would have liked to cover in this workshop, given more time, is **document-level comparisons**. There are two interesting methods in gensim, Word Mover's Distance and Doc2Vec, which do pretty different things. I'm happy to discuss those methods during the Q&A. 

### Further reading

I've loosely categorized some relevant papers into three groups. I'd recommend starting with papers in the first category, though there are some interesting methodological ideas in the second and third categories.

You'll notice that **time** is the covariate of choice for many of the applied papers. I'm excited to see what other sources of variation researchers can come up with. 

Social science papers: 

- Kozlowski et al 2019 ("Geometry of Culture") ([GitHub](https://github.com/KnowledgeLab/GeometryofCulture/))
- Stoltz and Taylor 2019, Taylor and Stoltz 2020a, Taylor and Stoltz 2020b, Stoltz and Taylor forthcoming (Concept Mover's Distance and extensions) ([GitHub](https://github.com/dustinstoltz/CMDist/))
- Jones et al 2019 (gender stereotypes decrease over time) ([GitHub](https://github.com/ruhulsbu/StereotypicalGenderAssociationsInLanguage))
- Rheault and Cochrane 2020 (ideology and parliamentary corporas) ([GitHub](https://github.com/lrheault/partyembed))
- Rodriguez and Spirling 2020 (methodological comparisons for political science research) ([GitHub](https://github.com/ArthurSpirling/EmbeddingsPaper))
- Arseniev-Koehler and Foster 2020 (cultural learning and what it means to be fat) ([GitHub](https://github.com/arsena-k/Word2Vec-bias-extraction))
- Nelson 2021 (machine learning and intersectionality) ([GitHub](https://github.com/lknelson/measuring_intersectionality))

Social application papers from NLP/CS researchers: 

- Bamman et al 2014 (embedding decomposition and geographic variation) ([GitHub](https://github.com/dbamman/geoSGLM))
- Kulkarni et al 2015 (historical semantic change over time) ([GitHub](https://github.com/viveksck/langchangetrack))
- Hamilton et al 2016a, 2016b (histwords - semantic change) ([website](https://nlp.stanford.edu/projects/histwords/))
- Garg et al 2017 (stereotypes and semantic change) ([GitHub](https://github.com/nikhgarg/EmbeddingDynamicStereotypes))
- Gonen and Goldberg 2019 ("Lipstick on a pig" - "debiasing" embeddings in terms of gender) ([GitHub](https://github.com/gonenhila/gender_bias_lipstick))
- Giulianelli et al 2020 (contextual embeddings for semantic change) ([GitHub](https://github.com/glnmario/cwr4lsc))
- Mendelsohn et al 2020 (dehumanization and linguistic change)
- Waller and Anderson 2020 (community embeddings)
- Soni et al 2021 (language change in abolitionist newspapers) ([GitHub](https://github.com/sandeepsoni/semantic-leadership-network))

Fundamental NLP/CS papers:

- Mikolov et al 2013 (word2vec) ([gensim tutorial](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html))
- Pennington et al 2014 (GloVe) ([website](https://nlp.stanford.edu/projects/glove/))
- Dai and Le 2015 (paragraph vectors / doc2vec) ([gensim tutorial](https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html))
- Kusner et al 2015 (Word Mover's Distance) ([gensim tutorial](https://radimrehurek.com/gensim/auto_examples/tutorials/run_wmd.html))
- Antoniak and Mimno 2018 (stability of embeddings) ([GitHub](https://github.com/maria-antoniak/word-embedding-stability))
- Sia et al 2020 (embeddings as topic models) ([GitHub](https://github.com/adalmia96/Cluster-Analysis))
- Warmerdam et al 2020 (whatlies package) ([website](https://rasahq.github.io/whatlies/))

## Contact information and acknowledgments

Connor Gilroy  
email: cgilroy at uw dot edu  
twitter: @ccgilroy

My thinking on word embeddings is indebted to SICSS-2017, a 2018 Text as Data course with John Wilkerson, many conversations (with Kate Stovel, Jeff Lockhart, Ian Kennedy, Nga Than, and others), and the code and papers listed above.

My research is partially supported by several grants, including an NIH NICHD training grant (T32 HD101442-01) to CSDE at the University of Washington and ARO Grant W911NF-19-1-0407.